## Problem 1 (50 pts)
This problem walks us through a problem discussed in detail in the Multi-level regression book
written by A. Gelman and J. Hill. NYC has a program known as stop-and-frisk. Relying on a
60’s era ruling, the law allows an officer to search someone without arrest, and without probable
cause, if the officer believes s/he might be in danger because of a hidden weapon. Much has
been written about this, as it has come under significant scrutiny for being discriminative and
allowing (even encouraging) racial profiling. You can read a summary of it at this Wikipedia page:
https://en.wikipedia.org/wiki/Stop-and-frisk_in_New_York_City.
The data you will download contain information about the number of traffic stops, reported per
precinct (75 total precincts), along with the ethnicity as reported by the police officer. These data
have kept only three ethnicities: white, black and hispanic. The data set also has data on arrest
rates in the previous year, broken down by four types of crimes; and the total population levels per
precinct per ethnic group.
1. Download and load the data in the file NYC stop and frisk.dat, uploaded to Canvas.
2. What fraction of the total stops correspond to “white/back/hispanic”? What fraction of the
population corresponds to “white/black/hispanic”?
3. Use a Poisson regression to model the number of stops, controlling for ethnicity and using
the number of past arrests as an exposure input.1
4. According to the output of your model, what fraction fewer or more stops does each ethnicity
have with respect to the others, in proportion to arrest rates of the previous year? Note that
you can just pick a baseline ethnicity and just compare everything to that.
5. Next, add the 75 precincts, and again solve the Poisson regression model.
6. Now, controlling for precincts, according to your model, what fraction fewer or more stops
does each ethnicity have with respect to the others, in proportion to arrest rates of the
previous year? (Again, just report with respect to a chosen ethnicity as a baseline).

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:
# !pip install pymc3
!pip install -U statsmodels


Requirement already up-to-date: statsmodels in /usr/local/lib/python3.6/dist-packages (0.10.1)


In [0]:
path = '/content/gdrive/My Drive/Colab Notebooks/TA/'

In [0]:
import numpy as np
import seaborn as sns
import pandas as pd
from pandas import DataFrame
import pymc3 as pm
import statsmodels.genmod.generalized_linear_model as sm
import statsmodels.genmod.families.family as family
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import math
import warnings
warnings.filterwarnings('ignore')


In [0]:
data = pd.read_csv(path+"NYC_stop_and_frisk.dat", sep = " ", header = 4)

total_stops = 0
total_pop = 0
stops_by_eth = [0,0,0]
pop_by_eth = [0,0,0]

for r in range(data.shape[0]):
    total_stops+=data['stops'][r]
    stops_by_eth[data['eth'][r] - 1] += data['stops'][r]
    if r%4==0:
        pop_by_eth[data['eth'][r] - 1] += data['pop'][r]
        total_pop += data['pop'][r]

print('Fraction of stops:')
print('black: %f, white: %f, hispanic: %f'%(stops_by_eth[0]/total_stops,stops_by_eth[1]/total_stops,stops_by_eth[2]/total_stops))

print('\nFraction of population:')
print('black: %f, white: %f, hispanic: %f'%(pop_by_eth[0]/total_pop,pop_by_eth[1]/total_pop,pop_by_eth[2]/total_pop))


Fraction of stops:
black: 0.531297, white: 0.339545, hispanic: 0.129158

Fraction of population:
black: 0.275469, white: 0.255741, hispanic: 0.468789


In [0]:
eths = pd.get_dummies(data['eth'])
eths.rename(columns={1: 'black', 2: 'hispanic,', 3: 'white'})
precincts = pd.get_dummies(data['precinct'])

for i in range(0,data['stops'].shape[0], 4):
    data.set_value(i, 'stops', data['stops'][i] + data['stops'][i+1] + data['stops'][i+2] + data['stops'][i+3])
    data.set_value(i, 'past.arrests', data['past.arrests'][i]+data['past.arrests'][i+1]+data['past.arrests'][i+2]+ data['past.arrests'][i+3])
    
data1 = pd.concat([data, eths], axis = 1)

In [0]:
data_cond = data1[data1.index % 4 == 0]
data_cond.head()

,stops,pop,past.arrests,precinct,eth,crime,1,2,3
0,202,1720,980,1,1,1,1,0,0
4,102,1368,295,1,2,1,0,1,0
8,81,23854,381,1,3,1,0,0,1
12,132,2596,753,2,1,1,1,0,0
16,144,6844,557,2,2,1,0,1,0


In [0]:
X = data_cond.drop(columns=['eth','crime','precinct', 'stops', 'past.arrests']).to_numpy()
y = data_cond['stops'].to_numpy()
exposure = data_cond['past.arrests'].to_numpy()
model = sm.GLM(endog = y, exog = X, family = family.Poisson(), exposure = exposure).fit()

print(model.summary())
print(model.params)

print('\nFor every white person stopped, %f black people are stopped'%(math.exp(model.params[1]-model.params[3])))
print('For every white person stopped, %f hispanic people are stopped'%(math.exp(model.params[2]-model.params[3])))

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  225
Model:                            GLM   Df Residuals:                      221
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -23511.
Date:                Thu, 07 Nov 2019   Deviance:                       45315.
Time:                        04:30:28   Pearson chi2:                 4.96e+04
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1          8.989e-07    8.1e-08     11.100      0.0

In [0]:
data1 = pd.concat([data1, precincts], axis =1)
data_cond_w_prec = data1[data1.index % 4 == 0]
X_with_prec = data_cond_w_prec.drop(columns = ['eth','crime','precinct', 'stops', 'past.arrests']).to_numpy()

model_w_prec = sm.GLM(endog = y, exog = X_with_prec, family = family.Poisson(), exposure = exposure).fit()

print(model_w_prec.summary())
print(model_w_prec.params)


                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                  225
Model:                            GLM   Df Residuals:                      147
Model Family:                 Poisson   Df Model:                           77
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2194.2
Date:                Thu, 07 Nov 2019   Deviance:                       2681.9
Time:                        04:37:50   Pearson chi2:                 2.73e+03
No. Iterations:                    40                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1          3.795e-06   1.42e-07     26.748      0.0

In [0]:

print('For every white person stopped, %f black people are stopped'%(math.exp(model_w_prec.params[1]-model_w_prec.params[3])))
print('For every white person stopped, %f hispanic people are stopped'%(math.exp(model_w_prec.params[2]-model_w_prec.params[3])))

For every white person stopped, 1.781774 black people are stopped
For every white person stopped, 1.759461 hispanic people are stopped


## Problem 2 (50 pts)
In this problem you will play with the idea of compound models. I have created a data set (entirely fake!)2 of 2012 salaries in the NBA, of 10,000 basketball players that were in high-school in 2011: nba cc fake data.csv. Note that the vast majority of the salaries are equal to 0 because the vast majority of these high-school players did not make it to the NBA and hence their NBA salary equals zero. There are three features you will use: height (in inches), average points scored during the last year in high school competition, and a scoring from 1-10 of the competitiveness of the league these players played in, with 10 being the most competitive.
The goal is to build a model to predict the NBA salary of a high school baller.

• Explain why linear regression is not appropriate, given the nature of the data.

• Try least squares regression, anyway. How well do you do?

• You will next build a composite model. You will first predict the probability that a player actually makes it to the NBA at all, and then you will build a model to predict the salary of a player, conditioned on the fact of making it to the NBA.

  – Build a model that predicts the probability of making it to the NBA.
  
  – Do a train-test split of 8000/2000 points, train your best model on the training set, and compute the AUC on the test set.
  
  – Now, build a model to predict the salary. Note that you may wish to consider a nonlinear transformation of your data. What is your R2 score on the test set?

• Compute the expected NBA salary of a high school basketball player who is 6’ 6” tall, is averaging 46 points per game, and is playing in the second most competitive league (comp =9), according to your model.

In [0]:
data = pd.read_csv(path+'nba_cc_fake_data.csv')
data.head()

,Unnamed: 0,Comp,Height,Points,Salary
0,0,9.0,76.0,27.0,0.0
1,1,7.0,78.0,39.0,0.0
2,2,9.0,76.0,39.0,0.0
3,3,9.0,74.0,39.0,0.0
4,4,9.0,74.0,26.0,0.0


In [0]:
data_arr = data.to_numpy()[:,1:]

X = data_arr[:,:-1]
Y = data_arr[:, -1:]

inverse = np.matmul(X.T,X)
calculated = np.matmul(np.linalg.inv(inverse), X.T)
beta_hat = np.matmul(calculated, Y)
r2score = r2_score(Y, np.matmul(X, beta_hat))
r2score

0.11688747097610919

The data is partially categorical. Linear regression is not appropriate since theree isn't a linear relationship between the features. The data is skewed with players who didn't make it to the NBA. Futher LSR r^2 value of 0.1 indicated the model is not good for predictions. 

In [0]:
data_nba = []

for row in data_arr:
  if row[3].item() > 0:
    data_nba.append(row)

data_nba = np.array(data_nba)
X_nba = data_nba[:,:-1]
Y_nba = data_nba[:,-1:]

inverse = np.matmul(X_nba.T,X_nba)
calculated = np.matmul(np.linalg.inv(inverse), X_nba.T)
beta_hat = np.matmul(calculated, Y_nba)

class_Y = []
for row in Y:
  if row>0:
    class_Y.append([1.])
  else:
    class_Y.append([row.item()])

class_Y = np.array(class_Y)

x_train, x_test, y_train, y_test = train_test_split(X, class_Y, test_size=0.2, random_state=0)
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict_proba(x_test)[:, 1:2]
pred = model.predict_proba([[9.,78.,46.]])

print('Probability that specified high school player makes it to the NBA %f'%(pred[0][1]))

fpr, tpr, threshold = roc_curve(y_test,y_pred, pos_label=1)
auc_roc = auc(fpr,tpr)
print('AUC value: %f'%(auc_roc))


Probability that specified high school player makes it to the NBA 0.060419
AUC value: 0.913313


In [0]:
class_Y = []
for row in Y:
  if row>0:
    class_Y.append([row.item()])
class_Y = np.array(class_Y)

x_train, x_test, y_train, y_test = train_test_split(X_nba, class_Y, test_size=0.2, random_state=0)
model = LogisticRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
pred = model.predict([[9.,78.,46.]])

print('Specified high school player\'s salary %f'%(int(pred[0])))
print('r2 score: %f'%(r2_score(y_test, y_pred)))



Specified high school player's salary 336837.000000
r2 score: -0.391554
